In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

nx = 9
ny = 6

In [2]:
def cal_undistort(img, objpoints, imgpoints):
    # Use cv2.calibrateCamera() and cv2.undistort()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

In [3]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

print(images)

# Step through the list and search for chessboard corners 
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        #img = cv2.drawChessboardCorners(img, (nx,ny), corners, ret)
        #cv2.imshow('img',img)
        #cv2.waitKey(1)


ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# print("mtx: " ,mtx)
# print("dist: " ,dist)

# Read in an image
img = cv2.imread('camera_cal/calibration1.jpg')
undistorted = cal_undistort(img, objpoints, imgpoints)

cv2.imshow('img',undistorted)
cv2.waitKey(1000)
print("undistorted")
#cv2.destroyAllWindows()


['camera_cal\\calibration1.jpg', 'camera_cal\\calibration10.jpg', 'camera_cal\\calibration11.jpg', 'camera_cal\\calibration12.jpg', 'camera_cal\\calibration13.jpg', 'camera_cal\\calibration14.jpg', 'camera_cal\\calibration15.jpg', 'camera_cal\\calibration16.jpg', 'camera_cal\\calibration17.jpg', 'camera_cal\\calibration18.jpg', 'camera_cal\\calibration19.jpg', 'camera_cal\\calibration2.jpg', 'camera_cal\\calibration20.jpg', 'camera_cal\\calibration3.jpg', 'camera_cal\\calibration4.jpg', 'camera_cal\\calibration5.jpg', 'camera_cal\\calibration6.jpg', 'camera_cal\\calibration7.jpg', 'camera_cal\\calibration8.jpg', 'camera_cal\\calibration9.jpg']
undistorted


In [4]:
def corners_unwarp(undistorted_img, nx, ny, mtx, dist):
    # 3) Find the chessboard corners
    ret= True
    print("img.shape[0]", undistorted_img.shape[0])
    print("img.shape[1]", undistorted_img.shape[1])
    src  =  np.float32([[0, undistorted_img.shape[0]//2], [undistorted_img.shape[1], undistorted_img.shape[0]//2], [undistorted_img.shape[1], undistorted_img.shape[0]], [0, undistorted_img.shape[0]] ])
    # 4) If corners found: 
    if ret:
        # c) define 4 destination points dst = np.float32([[,],[,],[,],[,]])

        #dst = np.float32([[0,0],[undistorted_img.shape[1]-100,100],[undistorted_img.shape[1]-100,undistorted_img.shape[0]-100],[100,undistorted_img.shape[0]-100]])
        dst = np.float32([[0,0],[undistorted_img.shape[1], 0], [undistorted_img.shape[1], undistorted_img.shape[0]],[0, undistorted_img.shape[0]]])
        # d) use cv2.getPerspectiveTransform() to get M, the transform matrix
        M = cv2.getPerspectiveTransform(src, dst)
        # e) use cv2.warpPerspective() to warp your image to a top-down view
        warped = cv2.warpPerspective(undistorted, M, (img.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR)
    #delete the next two lines
    return warped, M

In [5]:
img = cv2.imread('test_images/straight_lines1.jpg')
undistorted = cal_undistort(img, objpoints, imgpoints)

undistorted_gray = cv2.cvtColor(undistorted,cv2.COLOR_BGR2GRAY)
top_down, perspective_M = corners_unwarp(undistorted_gray, nx, ny, mtx, dist)
print("warped")
cv2.imshow('img',top_down)
cv2.waitKey(1000)

img.shape[0] 720
img.shape[1] 1280
warped


-1